<a href="https://colab.research.google.com/github/ray-hu/EM-Segmentation/blob/master/SVM_RF_EM_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-peparation 

### Import Libraries

In [4]:
# install
!pip install mahotas
!pip install progressbar2

In [0]:
import cv2
import numpy as np
import pylab as plt
from glob import glob
import argparse
import os
import progressbar
import pickle as pkl
from numpy.lib import stride_tricks
from skimage import feature
from sklearn import metrics
from sklearn.model_selection import train_test_split
import time
import mahotas as mt

### Mounting Google Drive locally

In [1]:
from google.colab import drive
drive.mount('./GoogleDrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./GoogleDrive


In [0]:
! ln -s GoogleDrive/My\ Drive ./drive

In [0]:
! ls

# Extract Features from the images

**Feature Vector:**


---


1.  ** Spectral(RGB)**


*   Red
*   Green
*   Blue


2.   **Texture**


*   Local binary pattern(LBP)



3.  ** Haralick's texture features**


*  Angular second moment
* Contrast
* Correlation
* Sum of Square: variance
* Inverse difference moment
* Sum average
* Sum variance
* Sum entropy
* Entropy


---

### Load dataset

In [62]:
import numpy as np
#from skimage import io
import glob
import cv2
import matplotlib.pyplot as plt

def load_data_set(load_aug = False):
  if load_aug:
    imgs_p = glob.glob("drive/data/aug/train/*.tif")
    labels_p = glob.glob("drive/data/aug/label/*.tif")
    tests_p = glob.glob("drive/data/test/*.tif")
  else:
    imgs_p = glob.glob("drive/data/train/*.tif")
    labels_p = glob.glob("drive/data/label/*.tif")
    tests_p = glob.glob("drive/data/test/*.tif")
  imgs_train = []
  label_train = []
  imgs_test = []
  print("_"*30)
  print("[INFO] Load orginal training data...")
  for img_p in imgs_p:
    imgs_train.append(cv2.imread(img_p, 1))
  for label_p in labels_p:
    label_train.append(cv2.imread(label_p,0))
    
  print("check the consistency of training data...")
  print("num of imgs_train: {}".format(len(imgs_train)))
  print('num of groundTruth_train: {}'.format(len(label_train)))
  print("_"*30)
  print("[INFO] Load orginal testing data...")
  for test_p in tests_p:
    imgs_test.append(cv2.imread(test_p,1))
  print('num of test images: {}'.format(len(imgs_test)))
  return imgs_train, label_train, imgs_test

imgs_train, label_train, imgs_test = load_data_set()

______________________________
[INFO] Load orginal training data...
check the consistency of training data...
num of imgs_train: 30
num of groundTruth_train: 30
______________________________
[INFO] Load orginal testing data...
num of test images: 30


### Extract Features

In [0]:
from skimage import feature
from numpy.lib import stride_tricks
import mahotas as mt
import progressbar

def extract_features(img, img_gary, label, train):
  #1.  RGB values have beed extracted when loading the orignal dataset
  
  #2.  Local Binary Pattern
  def LBP(img,points,radius):
    '''
     points: Number of circularly symmetric neighbour set points 
     radius: Radius of circle
    '''
    print(' extracting local binary pattern features.')
    lbp = feature.local_binary_pattern(img, points, radius)
    return (lbp-np.min(lbp))/(np.max(lbp)-np.min(lbp)) * 255
  
  #3. Haralick's texture features
  def HTF(img, h_neighbors, ss_dix):
    '''
    h_neighbors:
    '''
    print(' extracting haralick texture features.')
    size = h_neighbors
    shape = (img.shape[0] - size + 1, img.shape[1] - size + 1, size, size)
    strides = 2 * img.strides
    patches = stride_tricks.as_strided(img, shape=shape, strides = strides)
    patches = patches.reshape(-1,size,size)
    
    if len(ss_idx) == 0:
      bar = progressbar.ProgressBar(maxval = len(patches), \
                                   widgets = [progressbar.Bar('=','[', ']'), ' ', progressbar.Percentage()])
    else:
      bar = progressbar.ProgressBar(maxval=len(ss_idx), \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    
    
    def calculate_haralick(img):
      '''
      calculate haralick features for each patch
      '''
      features = []
      
      feature_ = mt.features.haralick(img)
      mean_ = feature_.mean(axis=0)
      
      [features.append(i) for i in mean_[0:9]]
      
      return np.array(features)
    
   
    bar.start()
    h_features = []
    
    if len(ss_idx) == 0:
      for i, p in enumerate(patches):
        bar.update(i+1)
        h_features.append(calculate_haralick(p))
    else:
      for i, p in enumerate(patches[ss_idx]):
        bar.update(i+1)
        h_features.append(calculate_haralick(p))
    
    return np.array(h_features)
  
  
  #4. extracting all the features
  
  # Hyperparameters:
  lbp_radius = 24 # local binary pattern neighbourhood
  lbp_points = lbp_radius*8 # number of circularly symmetric neighbour set points 
  h_neighbors = 11 # haralick neighbourhood
  
  
  num_examples = 1000 # number of examples per image to use for training model
  h_index = int((h_neighbors -1)/2)
  
  feature_img = np.zeros((img.shape[0], img.shape[1],4))
  feature_img[:,:,:3] = img
  feature_img[:,:,3] = LBP(img_gary, lbp_points, lbp_radius)
  feature_img = feature_img[h_index:-h_index, h_index:-h_index]
  s = feature_img.shape
  features = feature_img.reshape((s[0]*s[1], s[2]))
  
  if train:
    ss_idx = np.random.randint(0,features.shape[0],num_examples)
    features = features[ss_idx]
  else:
    ss_idx = []
  
  h_features = HTF(img_gary, h_neighbors, ss_idx)
  features = np.hstack((features, h_features))
  
  if train:
    label = label[h_index:-h_index, h_index:-h_index]
    labels = label.reshape(label.shape[0]*label.shape[1],1)
    labels = labels[ss_idx]
  else:
    labels = None
 
  return features, labels

### Generate new training dataset

In [0]:
def generate_training_data(imgs_train, label_train,train):
  print("_"*30)
  print("[INFO] Extracting features from original image(s)")
  
  Features = []
  Labels = []
  
  for i, img in enumerate(imgs_train):
    img_gary = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features, labels = extract_features(img,img_gary,label_train[i], train = train)
    Features.append(features)
    Labels.append(labels)
    
  Features = np.array(Features)
  s = Features.shape
  Features = Features.reshape((s[0]*s[1],s[2]))
  Labels = np.array(Labels)
  s = Labels.shape
  Labels = Labels.reshape((s[0]*s[1],s[2]))
  
  print("_"*30)
  print("[INFO] check the size of generated training data")
  print('size of the feature vector: {}'.format(Features.shape))
  print('size of the label vertor: {}'.format(Labels.shape))
  
  return Features, Labels
  

# Training

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

def train(train_X, train_Y, classifier):
  
  if classifier == "SVM":
    print("_"*30)
    print ('[INFO] Training a Support Vector Machine model.')
    model = SVC()
  elif classifier == "RF":
    print("_"*30)
    print('[INFO] Training a Random Forest model.')
    model = RandomForestClassifier(n_estimators=250, max_depth=12, random_state=42)
  elif classifier == "GBC":
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
    
    
  model.fit(train_X, train_Y)
  filename = 'drive/data/'+ classifier + '_model.pkl'
  with open(filename, 'wb') as file:  
    pickle.dump(model, file)
  
  print("_"*30)
  print('[INFO] Training complete.')
  print('training accuracy: %.f' %model.score(train_X,train_Y))
  
  return model

In [0]:
imgs_train, label_train, imgs_test = load_data_set()

In [63]:
label= np.array(label_train[0])
label.shape

(512, 512)

In [66]:
X, Y = generate_training_data(imgs_train, label_train,train=True)

______________________________
[INFO] Extracting features from original image(s)


[========================================================================] 100%

______________________________
[INFO] check the size of generated training data
size of the feature vector: (30000, 13)
size of the label vertor: (30000, 1)


In [0]:
train_X, test_X, train_Y, test_Y = train_test_split(X,Y,test_size=0.2, random_state=42) 

In [79]:
model = train(train_X, train_Y, classifier='RF')

[INFO] Training a Random Forest model.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


[INFO] Training complete.
training accuracy: 1


# Testing

In [0]:
def test(test_X, test_Y, classifier):
  
  # load trained model
  pkl_filename = filename = 'drive/data/'+ classifier + '_model.pkl'
  with open(pkl_filename, 'rb') as file:  
    model = pickle.load(file)
  
  # predict
  pred = model.predict(test_X)
  precision = metrics.precision_score(test_Y, pred, average='weighted', labels=np.unique(pred))
  recall = metrics.recall_score(test_Y, pred, average='weighted', labels=np.unique(pred))
  f1 = metrics.f1_score(test_Y, pred, average='weighted', labels=np.unique(pred))
  accuracy = metrics.accuracy_score(test_Y, pred)
  print ('_'*30)
  print('[RESULTS] Testing...')
  print ('Accuracy: %.2f' %accuracy)
  print ('Precision: %.2f' %precision)
  print ('Recall: %.2f' %recall)
  print ('F1: %.2f' %f1)
 

In [0]:
test(test_X, test_Y, classifier = 'SVM')

# Tuning Hyperparameters

#### Hyperparameter for SVM
1. C: regularization patameter
2. kernel: 'linear', 'poly', 'rbf', sigmoid, ...
3. degree: degree of 'poly' kernel
4. gamma: kernel coefficient for 'rbf', 'poly' and 'sigmoid'

#### Hyperparameter for RandomForest
1. n_estimators: num of trees in the forest
2. criterion: function to measure the quality of a split, 'gini' or 'entropy'
3. max_depth: max depth of the tree
4. min_samples_split: min num of samples to split an internal node
5. min_samples_leaf: min num of samples to be at a leaf node
6. max_features: num of features to consider when looking for the best split
7. bootstrap: method for sampling data points(with or without replacement)

#### Hyperparameter for feature extraction
1. lbp_radius: local binary pattern neighborhood
2. h_neighbors: haralick neighborhood
3. num_examples: num of examples per image to use for training

#### Other parameters related to above
1. lbp_points = lbp_radius*8
2. h_index = int((h_neighbors -1)/2)

## Tuning hyperparameters for RandomForest

In [0]:
import numpy as np
import pprint
from sklearn.model_selection import RandomizedSearchCV

def train_tune_para_RF(train_X, train_Y):
  
  print("_"*30)
  print('[INFO] Tuning hyperparameters for randomforest model.')
  
  # Number of trees in random forest
  n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
  # Criterion
  criterion = ['gini', 'entropy']
  # Number of features to consider at every split
  max_features = ['auto', 'sqrt']
  # Maximum number of levels in tree
  max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
  max_depth.append(None)
  # Minimum number of samples required to split a node
  min_samples_split = [2, 5, 10]
  # Minimum number of samples required at each leaf node
  min_samples_leaf = [1, 2, 4]
  # Method of selecting samples for training each tree
  bootstrap = [True, False]
  # Create the random grid
  random_grid = {'n_estimators': n_estimators,
                 'max_features': max_features,
                 'max_depth': max_depth,
                 'min_samples_split': min_samples_split,
                 'min_samples_leaf': min_samples_leaf,
                 'bootstrap': bootstrap}
  pprint(random_grid)
  # Use the random grid to search for best hyperparameters
  # First create the base model to tune
  rf = RandomForestClassifier()
  # Random search of parameters, using 3 fold cross validation, 
  # search across 100 different combinations, and use all available cores
  rf_search = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
  # Fit the random search model
  rf_search.fit(train_X, train_Y)
  
  filename = 'drive/data/RF_search_model.pkl'
  with open(filename, 'wb') as file:
    pickle.dump(rf_search, file)
    
  pkl_filename = 'drive/data/RF_search_model.pkl' 
  with open(pkl_filename, 'rb') as file:
    rf_search_model = pickle.load(file)

  print("_"*30)
  print("[RESULTS] After tuning")
  print('best hyperparameters:')
  pprint(rf_search_model.best_params_)
  # use these model to train the model again or :
  best_model = rf_search_model.best_estimator_
  best_accuracy = evaluate(best_model, test_X, test_Y)
  print('accuracy = {}'.format(best_accuracy))
  
  return rf_search

## Tuning hyperparameters for SVM

In [0]:
import numpy as np
import pprint
from sklearn.model_selection import RandomizedSearchCV

def train_tune_para_SVM(train_X, train_Y):
  
  print("_"*30)
  print('[INFO] Tuning hyperparameters for SVM model.')
  
  # C: regularization patameter
  
  C = np.linspace(0,1,num=10)
  # kernel: 'linear', 'poly', 'rbf', sigmoid, ...
  kernel = ['linear', 'poly', 'rbf', 'sigmoid']
  # degree: degree of 'poly' kernel
  degree = [3,10,30,100,300]
  # gamma: kernel coefficient for 'rbf', 'poly' and 'sigmoid'
  
  # Create the random grid
  random_grid = {'C': C,
                 'kernel': kernel,
                 'degree': degree}
  pprint(random_grid)
  # Use the random grid to search for best hyperparameters
  # First create the base model to tune
  estimator = SVC()
  # Random search of parameters, using 3 fold cross validation, 
  # search across 100 different combinations, and use all available cores
  svc_search = RandomizedSearchCV(estimator = estimator, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
  # Fit the random search model
  svc_search.fit(train_X, train_Y)
  
  filename = 'drive/data/SVC_search_model.pkl'
  with open(pkl_filename, 'wb') as file:
    pickle.dump(svc_search, file)
  
  pkl_filename = 'drive/data/SVC_search_model.pkl' 
  with open(pkl_filename, 'rb') as file:
    svc_search_model = pickle.load(file)
    
  print("_"*30)
  print("[RESULTS] After tuning")
  print('best hyperparameters:')
  pprint(svc_search_model.best_params_)
  # use these model to train the model again or :
  best_model = svc_search_model.best_estimator_
  best_accuracy = evaluate(best_model, test_X, test_Y)
  print('accuracy = {}'.format(best_accuracy))
    
  return svc_search

# Inference

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

def inference(imgs_test, model_path, results_dir, h_neighbors):
  
  print("_"*30)
  print('[INFO] Doing inference on test images...')
  
  print('loading trained model.')
  model = pickle.load(open(model_path, 'rb'))
  
  border = int((h_neighbors-1)/2)
  
  print('infer for each image')
  for img in imgs_test:
    img = cv2.copyMakeBorder(img, top=border, bottom=border, \
                                  left=border, right=border, \
                                  borderType = cv2.BORDER_CONSTANT, \
                                  value=[0, 0, 0])

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    features, _ = extract_features(img, img_gary, label=None, train=False)
    
    prediction = model.predict(features.reshape((-1, features.shape[1])))
    size_pred = int(math.sqrt(features.shape[0]))
    pred_img = prediction.reshape(size_pred, size_pred)
    cv2.imwrite(results_dir+'/'+file.split('/')[-1], pred_img)
  
  print("_"*30)
  print('[INFO] Inference complete.')
  print('show the last test image and its prediction:')

  f, ax = plt.subplots(1,2)
  ax[0].imshow(img)
  ax[0].set_title('test image')
  ax[1].imshow(pred_img)
  ax[1].set_title('prediction')
  plt.show()

In [0]:
inference(imgs_test, model_path = 'drive/data/RF_model.pkl' , results_dir='drive/data/results', h_neighbors = best_h_neighbors)